# 第六章：预知（逐步思考）

- [课程内容](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:


# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## 课程内容

如果有人叫醒您并立即开始问您几个复杂的问题，要求您立即回答，您会怎么做？可能不如给您时间**先思考答案**那么好。

猜怎么着？AI 模型也是如此。

**给模型时间逐步思考有时会使模型更准确**，特别是对于复杂任务。然而，**只有大声思考才算数**。您不能要求模型思考但只输出答案——在这种情况下，实际上没有发生思考。

### 示例

让我们看一个简单的计算问题，这类问题最能展示**让模型逐步思考的重要性**。这个问题看似简单，但大模型做起来确不容易。

In [ ]:
# 提示
PROMPT = """197*971"""

# 打印模型的响应
print(get_completion(PROMPT))

为了改善模型的响应，让我们**允许模型在回答之前先思考**。我们通过明确说明模型应该采取的步骤来处理和思考其任务来做到这一点。结合一点角色提示，这使模型能够更深入地理解评论。

In [ ]:
# 提示
PROMPT = """计算197*971

请在 <analysis> 标签中逐步分析这个计算问题，然后给出你的结论。"""

# 打印模型的响应
print(get_completion(PROMPT))

看到区别了吗？通过要求模型逐步思考，它能够逐步进行分析，并打印出来，这样的方式有助于大模型做出正确的回答。

这比简单回答"给"或"不给"要深刻得多。让我们看另一个展示思考重要性的例子。

现在让我们要求模型逐步思考同样的问题。看看有什么区别：

In [ ]:
# 提示
PROMPT = "说出一部由1956年出生的演员主演的著名电影。"

# 打印模型的响应
print(get_completion(PROMPT))

让我们通过要求模型逐步思考来解决这个问题，这次使用 `<brainstorm>` 标签。

In [ ]:
# 提示
PROMPT = "说出一部由1956年出生的演员主演的著名电影。首先在 <brainstorm> 标签中头脑风暴一些演员和他们的出生年份，然后给出你的答案。"

# 打印模型的响应
print(get_completion(PROMPT))

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 6.1 - 分类邮件](#exercise-61---classifying-emails)
- [练习 6.2 - 邮件分类格式化](#exercise-62---email-classification-formatting)

### 练习 6.1 - 分类邮件
在这个练习中，我们将指导模型将邮件分类到以下类别：
- (A) 售前问题
- (B) 损坏或有缺陷的物品
- (C) 账单问题
- (D) 其他（请解释）

在练习的第一部分，更改 `PROMPT` 以**使模型输出正确的分类且仅输出分类**。您的答案需要**包括正确选择的字母（A - D），带括号，以及类别名称**。

参考 `EMAILS` 列表中每封邮件旁边的注释，了解该邮件应归类到哪个类别。

In [ ]:
# 包含可变内容占位符的提示模板
PROMPT = """请将此邮件分类为绿色或蓝色：{email}"""

# 模型响应的预填充（如果有）
PREFILL = ""

# 作为列表存储的变量内容
EMAILS = [
    "你好 -- 我的Mixmaster4000在操作时发出奇怪的噪音。它还有点烟雾味和塑料味，像燃烧的电子产品。我需要更换。", # (B) 损坏或有缺陷的物品
    "我可以用我的Mixmaster 4000来搅拌油漆吗，还是只能用来搅拌食物？", # (A) 售前问题 OR (D) 其他（请解释）
    "我已经等了4个月，取消后我的月费还没有停止！！到底怎么回事？？？", # (C) 账单问题
    "我怎么到这里的，我不擅长电脑。求助。" # (D) 其他（请解释）
]

# 正确分类存储为列表的列表，以适应每封邮件可能有多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 用于正则表达式评分的每个类别的字符串值字典
REGEX_CATEGORIES = {
    "A": "A\) 售前",
    "B": "B\) 损坏",
    "C": "C\) 账单",
    "D": "D\) 其他"
}

# 遍历邮件列表
for i,email in enumerate(EMAILS):
    
    # 将邮件文本替换到邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取模型的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 对模型的响应进行评分
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # 打印模型的响应
    print("--------------------------- 包含变量替换的完整提示 ---------------------------")
    print("用户输入")
    print(formatted_prompt)
    print("\n助手回应")
    print(PREFILL)
    print("\n------------------------------------- 模型的响应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("此练习是否正确解决:", grade, "\n\n\n\n\n\n")

--------------------------- 包含变量替换的完整提示 ---------------------------
用户输入
请将邮件你好 -- 我的Mixmaster4000在操作时发出奇怪的噪音。它还有点烟雾味和塑料味，像燃烧的电子产品。我需要更换。分类到以下类别：
- (A) 售前问题
- (B) 损坏或有缺陷的物品
- (C) 账单问题
- (D) 其他（请解释）

助手回应


------------------------------------- 模型的响应 -------------------------------------
根据邮件内容，你的问题应该分类为：
- (B) 损坏或有缺陷的物品

因为你描述了你的Mixmaster4000在操作时发出奇怪的噪音，并且有烟雾味和塑料味，这表明设备可能存在物理损坏或制造缺陷。因此，你需要更换的是一个可能已经损坏或有缺陷的产品。

------------------------------------------ 评分 ------------------------------------------
此练习是否正确解决: True 






--------------------------- 包含变量替换的完整提示 ---------------------------
用户输入
请将邮件我可以用我的Mixmaster 4000来搅拌油漆吗，还是只能用来搅拌食物？分类到以下类别：
- (A) 售前问题
- (B) 损坏或有缺陷的物品
- (C) 账单问题
- (D) 其他（请解释）

助手回应


------------------------------------- 模型的响应 -------------------------------------
该邮件询问的是关于Mixmaster 4000的使用范围，具体是询问是否可以用来搅拌油漆，还是只能用来搅拌食物。这属于产品功能和使用方法的咨询，因此应该归类为：

- (A) 售前问题

因为这是在购买或使用前对产品功能的疑问，而不是关于产品损坏、账单问题或售后服务的问题。

------------------------------------------ 评分 -----------------------

❓ 如果您需要提示，请运行下面的单元格！

In [49]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

The grading function in this exercise is looking for the correct categorization letter + the closing parentheses and the first letter of the name of the category, such as "C) B" or "B) B" etc.
Let's take this exercise step by step:										
1.	How will Claude know what categories you want to use? Tell it! Include the four categories you want directly in the prompt. Be sure to include the parenthetical letters as well for easy classification. Feel free to use XML tags to organize your prompt and make clear to Claude where the categories begin and end.									
2.	Try to cut down on superfluous text so that Claude immediately answers with the classification and ONLY the classification. There are several ways to do this, from speaking for Claude (providing anything from the beginning of the sentence to a single open parenthesis so that Claude knows you want the parenthetical letter as the first part of the answer) to telling Claude that you want the classification and only the classifica

仍然遇到困难？运行下面的单元格查看示例解决方案。

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### 练习 6.2 - 邮件分类格式化
在这个练习中，我们将优化上述提示的输出，以产生完全按照我们想要的格式的答案。

使用您喜欢的输出格式化技术，让模型仅将正确分类的字母包装在 `<answer></answer>` 标签中。例如，第一封邮件的答案应该包含确切的字符串 `<answer>B</answer>`。

如果您忘记了每封邮件对应的正确字母类别，请参考 `EMAILS` 列表中每封邮件旁边的注释。

In [51]:
# 包含可变内容占位符的提示模板
PROMPT = """请将此邮件分类为绿色或蓝色：{email}"""

# 模型响应的预填充（如果有）
PREFILL = ""

# 作为列表存储的变量内容
EMAILS = [
    "你好 -- 我的Mixmaster4000在操作时发出奇怪的噪音。它还有点烟雾味和塑料味，像燃烧的电子产品。我需要更换。", # (B) 损坏或有缺陷的物品
    "我可以用我的Mixmaster 4000来搅拌油漆吗，还是只能用来搅拌食物？", # (A) 售前问题 OR (D) 其他（请解释）
    "我已经等了4个月，取消后我的月费还没有停止！！到底怎么回事？？？", # (C) 账单问题
    "我怎么到这里的，我不擅长电脑。求助。" # (D) 其他（请解释）
]

# 正确分类存储为列表的列表，以适应每封邮件可能有多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 用于正则表达式评分的每个类别的字符串值字典
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# 遍历邮件列表
for i,email in enumerate(EMAILS):
    
    # 将邮件文本替换到邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取模型的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 对模型的响应进行评分
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # 打印模型的响应
    print("--------------------------- 包含变量替换的完整提示 ---------------------------")
    print("用户输入")
    print(formatted_prompt)
    print("\n助手回应")
    print(PREFILL)
    print("\n------------------------------------- 模型的响应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("此练习是否正确解决:", grade, "\n\n\n\n\n\n")

--------------------------- 包含变量替换的完整提示 ---------------------------
用户输入
请将此邮件分类为绿色或蓝色：你好 -- 我的Mixmaster4000在操作时发出奇怪的噪音。它还有点烟雾味和塑料味，像燃烧的电子产品。我需要更换。

助手回应


------------------------------------- 模型的响应 -------------------------------------
根据邮件的内容，这里并没有直接提到是绿色还是蓝色的分类。不过，从邮件的内容来看，用户报告了他们设备（Mixmaster4000）出现的问题，包括奇怪的噪音、烟雾味和塑料燃烧的味道。这种描述更倾向于需要紧急处理的技术支持请求。如果这里的“绿色”和“蓝色”是代表不同类型的请求或优先级，那么根据描述的问题严重性，这可能更适合被分类为“红色”（表示紧急）或“黄色”（表示需要尽快处理），但既然题目要求选择“绿色”或“蓝色”，并且没有提供具体的分类标准，我们可能需要假设一个情景。

如果“绿色”代表一般性问题或较低优先级的请求，而“蓝色”代表更紧急或需要立即关注的问题，那么根据邮件中描述的设备出现的严重问题（如奇怪的噪音、烟雾和燃烧的味道），这封邮件更可能被分类为“蓝色”，因为它暗示了设备可能存在严重的故障，需要尽快处理以避免进一步的损害或安全风险。

因此，根据上述分析，我会将此邮件分类为“蓝色”。

------------------------------------------ 评分 ------------------------------------------
此练习是否正确解决: False 






--------------------------- 包含变量替换的完整提示 ---------------------------
用户输入
请将此邮件分类为绿色或蓝色：我可以用我的Mixmaster 4000来搅拌油漆吗，还是只能用来搅拌食物？

助手回应


------------------------------------- 模型的响应 -------------------------------------
从邮件的内容来看，这封邮件提到了一个名为"Mixmaster 

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### 恭喜！

如果您已经解决了到此为止的所有练习，您就可以进入下一章了。祝您提示愉快！

---

## 示例练习场

这是一个供您自由试验本课程中显示的提示示例的区域，您可以调整提示以查看它如何影响模型的响应。

In [ ]:
# 提示
PROMPT = """197*971"""

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = """计算197*971

请在 <analysis> 标签中逐步分析这个计算问题，然后给出你的结论。"""

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "说出一部由1956年出生的演员主演的著名电影。"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "说出一部由1956年出生的演员主演的著名电影。首先在 <brainstorm> 标签中头脑风暴一些演员和他们的出生年份，然后给出你的答案。"

# 打印模型的响应
print(get_completion(PROMPT))